In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, ElementClickInterceptedException
import time
import requests

In [43]:
def find_sector_ids(url):
    """Function to scrap sector IDs in elements with onclick property from Banxico.

    Parameters
    ----------
    url : str
        banxico url containig the series id information

    Returns
    -------
    sector_ids : list
        list containing sector ids as strings
    """
    sector_ids = []
    try:
        driver = webdriver.Chrome()
        driver.get(url)

        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(5)
        
        #driver.find_element_by_xpath(".//input[contains(@onclick, '1 Bedroom Deluxe')]")
        elements = driver.find_elements(By.XPATH, '//*[@onclick]')

        if len(elements) < 3: 
            driver.quit()
            time.sleep(10)
            find_sector_ids(url)


        for element_id in elements:
            onclick_txt = element_id.get_attribute('onclick')
            if "cargaDirectorioSector" in onclick_txt:
                sector_ids.append((onclick_txt.split(",")[0].split("(")[-1]))
        return sector_ids

    finally:
        driver.quit()


banxico_series_url = "https://www.banxico.org.mx/SieAPIRest/service/v1/doc/catalogoSeries" 

sector_ids = find_sector_ids(banxico_series_url)
sector_ids


['4',
 '23',
 '6',
 '18',
 '22',
 '7',
 '3',
 '19',
 '11',
 '9',
 '8',
 '20',
 '2',
 '10',
 '1',
 '5',
 '21',
 '24',
 '25']

In [116]:
sectores_api_url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/cat/sectores/"
def find_cuadroid_by_sectorid(sectors_ids):
    id_cuadros = []

    def recursive_func(item):
        if isinstance(item, dict):
            if 'idCuadro' in item:
                id_cuadros.append(item['idCuadro'])
            if 'cuadros' in item:
                recursive_func(item['cuadros'])
        elif isinstance(item, list):
            for i_t in item:
                recursive_func(i_t)
                
    for sector_id in sectors_ids:
        api_url = sectores_api_url+sector_id
        response = requests.get(api_url, headers={"Accept": "application/json"})

        if response.status_code == 200:
            data = response.json()
            recursive_func(data['bmx']['cuadros']) 
        else:
            id_cuadros.append('')
    
    return id_cuadros
        

cuadros_ids = find_cuadroid_by_sectorid(sector_ids)

len(cuadros_ids)
cuadros_ids


['CF100',
 'CF119',
 'CF120',
 'CF104',
 'CF106',
 'CF2',
 'CF139',
 'CF105',
 'CF894',
 'CF1',
 'CF893',
 'CF169',
 'CF456',
 'CF791',
 'CF364',
 'CF365',
 'CF372',
 'CF375',
 'CF359',
 'CF366',
 'CF367',
 'CF892',
 'CF887',
 'CF901',
 'CF376',
 'CF102',
 'CF372',
 'CF373',
 'CF361',
 'CF374',
 'CF362',
 'CF210',
 'CF777',
 'CF782',
 'CF449',
 'CF450',
 'CF307',
 'CF86',
 'CF85',
 'CF336',
 'CF909',
 'CR60',
 'CF349',
 'CF300',
 'CF347',
 'CF114',
 'CF348',
 'CF302',
 'CF101',
 'CF370',
 'CF111',
 'CF113',
 'CF112',
 'CF117',
 'CF303',
 'CF840',
 'CF815',
 'CF870',
 'CI34',
 'CF907',
 'CF346',
 'CF904',
 'CF875',
 'CF108',
 'CF624',
 'CF107',
 'CF377',
 'CF908',
 'CF916',
 'CF447',
 'CF474',
 'CF115',
 'CF371',
 'CF133',
 'CF304',
 'CF577',
 'CF460',
 'CF457',
 'CF478',
 'CF70',
 'CF69',
 'CG11',
 'CF133',
 'CF301',
 'CF304',
 'CF103',
 'CF57',
 'CF56',
 'CF58',
 'CF448',
 'CF119',
 'CF120',
 'CF104',
 'CF106',
 'CF139',
 'CF2',
 'CF807',
 'CF886',
 'CF76',
 'CF77',
 'CF808',
 'CF834'